<a href="https://colab.research.google.com/github/yoheikikuta/US-patent-analysis/blob/master/colab/feature_extract_BERT_SecondLastReduceMean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Extraction from BERT model

Use the second last mean of the features.

In [0]:
from google.colab import auth
auth.authenticate_user()

W0807 13:09:10.675693 140446435125120 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Data preparation

Since the input of the BERT model is token-ids corresponding to tokens using the BERT tokenization, use the prepared tokenization.

In [0]:
DATA_DIR = "./"

class ModelInfo:
  gcs_model_path = "gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/"
  model_path = DATA_DIR + "uncased_L-12_H-768_A-12/"
  feature_dim = 768
  gcs_save_path = "gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/BERT-2ndlastmean/"
  
  ### For Large model.
#   gcs_model_path = "gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-24_H-1024_A-16/"
#   model_path = DATA_DIR + "uncased_L-24_H-1024_A-16/"
#   feature_dim = 1024
#   gcs_save_path = "gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/BERT-Large-2ndlastmean/"

In [0]:
!gsutil cp -r {ModelInfo.gcs_model_path} {DATA_DIR}

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_config.json...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt.index...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_model.ckpt.meta...
/ [4 files][420.9 MiB/420.9 MiB]   22.0 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/checkpoint...
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncas

In [0]:
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/grants_for_3000+3000.df.gz {DATA_DIR}
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/testset_app_3000.df.gz {DATA_DIR}
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/training_app_3000.df.gz {DATA_DIR}

Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/grants_for_3000+3000.df.gz...
/ [1 files][129.4 MiB/129.4 MiB]                                                
Operation completed over 1 objects/129.4 MiB.                                    
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/testset_app_3000.df.gz...
| [1 files][ 45.5 MiB/ 45.5 MiB]                                                
Operation completed over 1 objects/45.5 MiB.                                     
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/training_app_3000.df.gz...
/ [1 files][ 45.0 MiB/ 45.0 MiB]                                                
Operation completed over 1 objects/45.0 MiB.                                     


In [0]:
import gzip
import pickle
import datetime
import json
import os
import pprint
import random
import string
import sys
import pandas as pd

In [0]:
grants = pd.read_pickle(f"{DATA_DIR}grants_for_3000+3000.df.gz")
test_app = pd.read_pickle(f"{DATA_DIR}testset_app_3000.df.gz")
train_app = pd.read_pickle(f"{DATA_DIR}training_app_3000.df.gz")

In [0]:
train_app.head(3)

,app_id,xml
0,12130785,"<us-patent-application lang=""EN"" dtd-version=""..."
1,12652424,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12214532,"<us-patent-application lang=""EN"" dtd-version=""..."


In [0]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 279.30 KiB | 3.77 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [0]:
import sys
sys.path.append("./bert")

import tokenization

In [0]:
tokenizer = tokenization.FullTokenizer(
    vocab_file=f"{ModelInfo.model_path}vocab.txt", do_lower_case=True)

W0807 13:10:12.149398 140446435125120 deprecation_wrapper.py:119] From ./bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
def claim_to_ids(claim):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(claim))


import re


CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)
TAG_PAT = re.compile(r"<.*?>")
LB_PAT = re.compile(r'[\t\n\r\f\v][" "]*')
CANCELED_PAT = re.compile(r'[0-9]+.*\. \(canceled\)[" "]')
NUM_PAT = re.compile(r'[" "]?[0-9]+[" "]?\.[" "]?')

def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)
def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))

def remove_linebreak_from_claim(claim):
    return LB_PAT.sub('', claim)

def remove_canceled_claim(claim):
    return CANCELED_PAT.sub('', claim)
  
def remove_claim_numbers(claim):
    return NUM_PAT.sub('', claim)  

In [0]:
%%time

train_ids = [claim_to_ids(claim) for claim in train_app["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)]
test_ids = [claim_to_ids(claim) for claim in test_app["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)]
grants_ids = [claim_to_ids(claim) for claim in grants["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)]

CPU times: user 5min 35s, sys: 414 ms, total: 5min 35s
Wall time: 5min 35s


In [0]:
list(map(len, train_ids))[:10], list(map(len, test_ids))[:10], list(map(len, grants_ids))[:10]

([774, 1065, 1698, 486, 1863, 724, 711, 566, 1724, 1469],
 [3296, 3724, 2410, 295, 286, 216, 832, 407, 2111, 1121],
 [1491, 671, 417, 1441, 4625, 2778, 322, 643, 1016, 2518])

In [0]:
train_ids[0][:10], train_ids[1500][:10], train_ids[-1][:10]

([1037, 2291, 2005, 2367, 15370, 5005, 2013, 2019, 12098, 25032],
 [1037, 6276, 6994, 2383, 1037, 16305, 2364, 2303, 1998, 1037],
 [1037, 6381, 6121, 4653, 14709, 2008, 7711, 6381, 14438, 2007])

In [0]:
test_ids[0][:10], test_ids[1500][:10], test_ids[-1][:10]

([1037, 4118, 2005, 5464, 2019, 9178, 2114, 1037, 2177, 1997],
 [1037, 4118, 1997, 4488, 2019, 3169, 1999, 1037, 4942, 17310],
 [1037, 17880, 1011, 29014, 3252, 9605, 1024, 1037, 23604, 19395])

In [0]:
grants_ids[0][:10], grants_ids[1500][:10], grants_ids[-1][:10]

([1037, 4118, 2005, 12515, 2019, 4358, 10146, 1997, 1037, 8331],
 [2019, 9413, 7446, 22026, 18921, 6218, 4057, 3085, 2000, 1037],
 [1037, 4118, 9605, 1024, 4909, 1037, 5227, 2013, 1037, 7396])

## Feature extraction

In [0]:
import tensorflow as tf

tf.__version__

'1.14.0'

In [0]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.127.203.82:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 18155664102575052123),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 181401334104015189),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5097573642739295921),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5477618557185349819),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8972939774992622740),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 6818830325481115855),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 16118514186278571439),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5840110353621396503),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 65631391227031

In [0]:
CLS_ID = tokenizer.vocab["[CLS]"]
SEP_ID = tokenizer.vocab["[SEP]"]

max_seq_length = 512

In [0]:
CLS_ID, SEP_ID

(101, 102)

In [0]:
def convert_single_sentence(ids_a_input, uniqueue_id):
  ids_a = ids_a_input
  # Account for [CLS] and [SEP] with "- 2"
  if len(ids_a_input) > max_seq_length - 2:
    ids_a = list(ids_a_input)[0:(max_seq_length - 2)]  
      

  # The convention in BERT is:
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  input_ids = []
  segment_ids = []
  input_ids.append(CLS_ID)
  segment_ids.append(0)
  for token in ids_a:
    input_ids.append(token)
    segment_ids.append(0)
  input_ids.append(SEP_ID)
  segment_ids.append(0)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  return (input_ids, input_mask, segment_ids, uniqueue_id)

In [0]:
check = convert_single_sentence(train_ids[0], 0)

In [0]:
check[0][:5], check[1][:5], check[2][:5]

([101, 1037, 2291, 2005, 2367], [1, 1, 1, 1, 1], [0, 0, 0, 0, 0])

In [0]:
BASE_TRAINING=10000
BASE_TEST = 20000
BASE_GRANTS = 30000


# unique_id, tokens, input_ids, input_mask, input_type_ids
def feature_tuplist_to_feature_dict(ftups):
    dic = {}
    dic['input_ids'] = [tup[0] for tup in ftups]
    dic["input_mask"] = [tup[1] for tup in ftups]
    dic["input_type_ids"] = [tup[2] for tup in ftups]
    dic['unique_ids'] = [tup[3] for tup in ftups]
    return dic

In [0]:
check = feature_tuplist_to_feature_dict([check])

In [0]:
check['input_ids'][0][:5], check['input_mask'][0][:5], check['input_type_ids'][0][:5], check['unique_ids'][0]

([101, 1037, 2291, 2005, 2367], [1, 1, 1, 1, 1], [0, 0, 0, 0, 0], 0)

In [0]:
import modeling
from extract_features import model_fn_builder

In [0]:
bert_config = modeling.BertConfig.from_json_file(f"{ModelInfo.model_path}bert_config.json")

In [0]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.vocab_file = f"{ModelInfo.model_path}vocab.txt"
        self.use_tpu = True
        self.output_dir = "./"
        self.init_checkpoint = f"{ModelInfo.gcs_model_path}bert_model.ckpt"
#         self.init_checkpoint = "gs://mlstudy-phys/mlstudy-phys/bert/model/patent_1000/model.ckpt-1637"  # fine-tuned model
        self.predict_batch_size = 16
        # TPU related
        self.num_tpu_cores = 8
        self.tpu_name = TPU_ADDRESS
        
        # following parameters are not used anymore. (because we create feature by hand)
        self.do_lower_case = True
        self.max_seq_length = 512
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        # TPU related. Some of these value have positive int not to make TPUEstimator angry (even though these are not used...).
        self.eval_batch_size = 32
        self.train_batch_size = 32 
        self.iterations_per_loop = 1000

        
FLAGS = FLAGS()

In [0]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu_name)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

In [0]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=FLAGS.init_checkpoint,
    use_tpu=FLAGS.use_tpu,
    layer_indexes=[-2], # Use the second last hidden layer
    use_one_hot_embeddings=FLAGS.use_tpu)

In [0]:
# https://github.com/kyzhouhzau/BERT-NER/issues/19

def model_fn_wrapper(features, labels, mode, params):
  tspec = model_fn(features, labels, mode, params)
  # We use the second last layer!
  pred_dict = {'predictions': tspec.predictions['layer_output_0']}
  return tf.contrib.tpu.TPUEstimatorSpec(
          mode=tspec.mode, predictions=pred_dict, scaffold_fn=tspec.scaffold_fn)

In [0]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    # model_fn=model_fn,
    model_fn=model_fn_wrapper,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

W0807 13:15:51.379627 140446435125120 estimator.py:1984] Estimator's model_fn (<function model_fn_wrapper at 0x7fbbf47dad08>) includes params argument, but params are not passed to Estimator.


In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
def ids_to_input_fdict(ids, base_id):
  return feature_tuplist_to_feature_dict([convert_single_sentence(one_app, base_id+idx) for idx, one_app in enumerate(ids)])

def fdict_to_features(fdict):

  def _batch_input_function(params):
    test_ds = tf.data.Dataset.from_tensor_slices(fdict)    
    return test_ds.batch(params['batch_size'])

  feature_results =  np.empty((0, ModelInfo.feature_dim), float)
  print(datetime.datetime.today())
  result = estimator.predict(_batch_input_function, yield_single_examples=True)
  
  for idx, one in enumerate(result):
    # reduce mean.
    mask = fdict["input_mask"][idx]
    validlen = sum(mask)
    features = one['predictions'].reshape(512, ModelInfo.feature_dim).sum(axis=0) / validlen
    
    feature_results = np.append(feature_results, features.reshape(1, ModelInfo.feature_dim), axis=0)  
  print(datetime.datetime.today())
  return feature_results

In [0]:
GS_BASE = ModelInfo.gcs_save_path

TRAIN_FEATURE_FNAME = "bert_2ndlastmean_feature_train_app_3000.pkl.gz"
TEST_FEATURE_FNAME = "bert_2ndlastmean_feature_test_app_3000.pkl.gz"
GRANTS_FEATURE_FNAME = "bert_2ndlastmean_feature_grants_3000_3000.pkl.gz"

# TRAIN_FEATURE_FNAME = "bert_large_2ndlastmean_feature_train_app_3000.pkl.gz"
# TEST_FEATURE_FNAME = "bert_large_2ndlastmean_feature_test_app_3000.pkl.gz"
# GRANTS_FEATURE_FNAME = "bert_large_2ndlastmean_feature_grants_3000_3000.pkl.gz"

In [0]:
def dump_and_send(fname, obj):
  with gzip.open(fname, 'w') as f:
     pickle.dump(obj, f)
  !gsutil cp {fname} {GS_BASE}{fname}

In [0]:
import numpy as np

In [0]:
train_app_input_fdict = ids_to_input_fdict(train_ids, BASE_TRAINING)
train_features = fdict_to_features(train_app_input_fdict)

2019-08-07 13:15:52.440129
2019-08-07 13:17:12.904742


In [0]:
train_features[0, 0:10]

array([-0.61115211,  0.09057721,  0.14272423, -0.23332408,  0.42033839,
        0.1024638 ,  0.15721108, -0.2079207 ,  0.28475305,  0.24445903])

In [0]:
train_features.shape

(3000, 768)

In [0]:
dump_and_send(TRAIN_FEATURE_FNAME, train_features)

Copying file://bert_2ndlastmean_feature_train_app_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/9.4 MiB.                                      


In [0]:
test_app_input_fdict = ids_to_input_fdict(test_ids, BASE_TEST)
test_features = fdict_to_features(test_app_input_fdict)

2019-08-07 13:17:43.098740
2019-08-07 13:18:45.484968


In [0]:
test_features.shape

(3000, 768)

In [0]:
dump_and_send(TEST_FEATURE_FNAME, test_features)

Copying file://bert_2ndlastmean_feature_test_app_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/9.4 MiB.                                      


In [0]:
grants_input_fdict = ids_to_input_fdict(grants_ids, BASE_GRANTS)
grants_features = fdict_to_features(grants_input_fdict)

2019-08-07 13:19:16.868098
2019-08-07 13:21:42.594636


In [0]:
grants_features.shape

(6440, 768)

In [0]:
dump_and_send(GRANTS_FEATURE_FNAME, grants_features)

Copying file://bert_2ndlastmean_feature_grants_3000_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.1 MiB.                                     


# Trial and Errors

In [0]:
def ids_to_input_fdict(ids, base_id):
  return feature_tuplist_to_feature_dict([convert_single_sentence(one_app, base_id+idx) for idx, one_app in enumerate(ids)])

def fdict_to_features(fdict):

  def _batch_input_function(params):
    test_ds = tf.data.Dataset.from_tensor_slices(fdict)    
    return test_ds.batch(params['batch_size'])

  feature_results =  np.empty((0, ModelInfo.feature_dim), float)
  print(datetime.datetime.today())
  result = estimator.predict(_batch_input_function, yield_single_examples=True)
  return result
  
#   for idx, one in enumerate(result):
#     # reduce mean.
#     mask = fdict["input_mask"][idx]
#     validlen = sum(mask)
#     features = one['predictions'].reshape(512, ModelInfo.feature_dim).sum(axis=0) / validlen
    
#     feature_results = np.append(feature_results, features.reshape(1, ModelInfo.feature_dim), axis=0)  
#   print(datetime.datetime.today())
#   return feature_results

In [0]:
check_train_app_input_fdict = ids_to_input_fdict(train_ids[:100], BASE_TRAINING)

In [0]:
check_train_app_input_fdict['input_ids'][0][:5]

[101, 1037, 2291, 2005, 2367]

In [0]:
result = fdict_to_features(check_train_app_input_fdict)

2019-08-07 04:13:08.340518


In [0]:
one = next(result)

In [0]:
one['predictions'].shape

(512, 768)

In [0]:
one['predictions'][0][:5]

array([-0.399356  , -0.86181027, -0.35524815,  0.18730514,  0.3189999 ],
      dtype=float32)

EXPECTED: 

array([-0.14263938, -0.6473565 , -0.25649723,  0.19866335,  0.58390003],
      dtype=float32)